# Part 1 : Installing Darknet

## Import Drive Files


In [0]:
%cd /content/
### Move files to Drive -> Colab/content

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Google Authentification
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Download file key.cfg
## Lien : https://drive.google.com/open?id=1IzZZbZmsQuJLyTK7eaKYnVO94Yotv6q_
downloaded = drive.CreateFile({'id': '1IzZZbZmsQuJLyTK7eaKYnVO94Yotv6q_'})
downloaded.GetContentFile('key.cfg')

## Download file darknet.zip
## Lien : https://drive.google.com/open?id=1SHzfC5SoZb_fANtPGoQv80xTI2GEoQ55
downloaded = drive.CreateFile({'id': '1SHzfC5SoZb_fANtPGoQv80xTI2GEoQ55'})
downloaded.GetContentFile('darknet.zip')

## Download file keras_yolov3.zip
## Lien : https://drive.google.com/open?id=1CUVBT8SVsx04pDk3zbWe29kVtI0WpVCv
downloaded = drive.CreateFile({'id': '1CUVBT8SVsx04pDk3zbWe29kVtI0WpVCv'})
downloaded.GetContentFile('keras_yolov3.zip')

## Download file key_last.weights
## Lien : https://drive.google.com/open?id=1_VXc0GIkG3dtcs_ey6zffeWRXpGVZ3BH
downloaded = drive.CreateFile({'id': '1_VXc0GIkG3dtcs_ey6zffeWRXpGVZ3BH'})
downloaded.GetContentFile('key_last.weights')

## Download file key_last.weights
## Lien : https://drive.google.com/open?id=11_3Fo3bcurdaEdOosJXtp9ZO45vWtUNA
downloaded = drive.CreateFile({'id': '11_3Fo3bcurdaEdOosJXtp9ZO45vWtUNA'})
downloaded.GetContentFile('cudnn-10.0-linux-x64-v7.5.0.56.tgz')

!unzip darknet.zip
!unzip keras_yolov3.zip

## Install cuDNN

In [45]:
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you 
!/usr/local/cuda/bin/nvcc --version

# Got Cuda 10.0

# Download zip file of cuDNN

# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf /content/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed.
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.5.0
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


## Install Darknet

### Cloning and compiling Darknet

In [0]:
!git clone https://github.com/kriyeng/darknet/
%cd darknet

# Check the folder
#!ls

# I have a branch where I have done the changes commented above
#!git checkout feature/google-colab

#Compile Darknet
!make

#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/Défis\ et\ Séminaires\ en\ Intelligence\ Artificielle/Défi\ 2/darknet/bin/darknet

# Set execution permissions to Darknet
!chmod +x ./darknet

## Finish ! Test [Optionnal]

In [0]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path)

# Get yolov3 weights
!wget https://pjreddie.com/media/files/yolov3.weights

# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

### All works ? We can continue.

# Part 2 : Training with Yolo

## Loading data

In [0]:
cd /content/
!git clone https://github.com/belarbi2733/keras_yolov3
cd /content/keras_yolov3
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/key_wb.zip
!wget https://github.com/belarbi2733/keras_yolov3/releases/download/1/bckgrnd.zip
!unzip bckgrnd.zip 
!unzip key_wb.zip
!python keys_with_background.py --keys "key_wb" --background "bckgrnd" --output "/content/darknet/img"
!mv img/annotations.csv .
!wget https://pjreddie.com/media/files/yolov3.weights

/content


In [0]:
## Optionnal
## Show img files to put them in a txt file
#%%bash
#for file in '/content/darknet/img'/*
#do
#  echo $file
#done

## Training

In [0]:
%cd /content/darknet
# Download Darknet53.74
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [0]:
!mv img/annotations.csv .
!mv img/annotations.txt .
!mv img/annotation_python.py .
!mv img/annotation_python_v2.py .

In [53]:
# Training
%cd /content/darknet
!./darknet detector train cfg/key.data /content/key-5.cfg darknet53.conv.74 -dont_show

/content/darknet
key-5
layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32 0.639 BF
   1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64 3.407 BF
   2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32 0.379 BF
   3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64 3.407 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128 3.407 BF
   6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64 0.379 BF
   7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128 3.407 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64 0.379 BF
  10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128 3.407 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   152 x 152 x 128   ->    76 x  76 x 256 3.407 BF
  13 conv    128  1 x 1 / 1    76 x  76 x 256   ->    76 x  7

In [0]:
# Testing

def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)
  
%cd /content/darknet
!./darknet detector test cfg/key_voc.data /content/key.cfg /content/darknet/backup/key_last.weights /content/darknet/img/gen_0041.jpg #-thresh 0.49
# Show the result using the helper imgShow()
imShow('predictions.jpg')

In [0]:
# Precision
./darknet detector map data/key.data /content/key.cfg backup/key_last.weights